# Tiling into stripes

### table of content
1) [Stripping](#stripping)
2) [Merging of stripes](#merging-of-stripes)

### Dependencies and general utils

In [2]:
import os
import numpy as np
import pdal
import json
import laspy
from tqdm import tqdm

### Stripping

In [49]:
def stripes_file(src_input_file, src_output, tile_size_x, tile_size_y):
    laz = laspy.read(src_input_file)
    output_folder = os.path.join(src_output, f"{os.path.basename(src_input_file).split('.laz')[0]}_stripes_{tile_size_x}_{tile_size_y}")
    os.makedirs(output_folder, exist_ok=True)

    xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

    x_edges = np.arange(xmin, xmax, tile_size_x)
    y_edges = np.arange(ymin, ymax, tile_size_y)


    for i, x0 in enumerate(x_edges):
        print(f"Column {i+1} / {len(x_edges)}:")
        for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
            x1 = x0 + tile_size_x
            y1 = y0 + tile_size_y
            bounds = f"([{x0},{x1}],[{y0},{y1}])"
            output = os.path.join(output_folder,f"{os.path.basename(src_input_file).split('.laz')[0]}_stripe_{i}_{j}.laz")
            pipeline_json = {
                "pipeline": [
                    src_input_file,
                    {"type": "filters.crop", "bounds": bounds},
                    {"type": "writers.las", "filename": output}
                ]
            }

            pipeline = pdal.Pipeline(json.dumps(pipeline_json))
            pipeline.execute()


def stripes_file_with_overlap(src_input_file, src_output, tile_size_x, tile_size_y, overlap=0):
    assert overlap < tile_size_x
    assert overlap < tile_size_y
    
    laz = laspy.read(src_input_file)
    output_folder = os.path.join(src_output, f"{os.path.basename(src_input_file).split('.laz')[0]}_ov_{overlap}_stripes_{tile_size_x}_{tile_size_y}")
    os.makedirs(output_folder, exist_ok=True)

    xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

    x_edges = np.arange(xmin, xmax, tile_size_x)
    y_edges = np.arange(ymin, ymax, tile_size_y)


    for i, x0 in enumerate(x_edges):
        print(f"Column {i+1} / {len(x_edges)}:")
        for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
            x1 = x0 + tile_size_x
            y1 = y0 + tile_size_y

            # application of overlapp
            x0 = max(x0 - overlap / 2, xmin)
            x1 = min(x1 + overlap / 2, xmax)
            y0 = max(y0 - overlap / 2, ymin)
            y1 = min(y1 + overlap / 2, ymax)

            bounds = f"([{x0},{x1}],[{y0},{y1}])"
            output = os.path.join(output_folder,f"{os.path.basename(src_input_file).split('.laz')[0]}_stripe_{i}_{j}.laz")
            pipeline_json = {
                "pipeline": [
                    src_input_file,
                    {"type": "filters.crop", "bounds": bounds},
                    {"type": "writers.las", "filename": output}
                ]
            }

            pipeline = pdal.Pipeline(json.dumps(pipeline_json))
            pipeline.execute()

In [45]:
src_input_file = r"D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21.laz"
src_output = os.path.dirname(src_input_file)

tile_size_x, tile_size_y = 100, 20

In [46]:
# Show infos on tile
do_print_infos = False
if do_print_infos:
    laz = laspy.read(src_input_file)
    print("Min x : ", np.min(laz.x))
    print("Max x : ", np.max(laz.x))
    print("Min y : ", np.min(laz.y))
    print("Max y : ", np.max(laz.y))
    delta_x = np.max(laz.x) - np.min(laz.x)
    delta_y = np.max(laz.y) - np.min(laz.y)
    print("---")
    print("delta x : ", delta_x)
    print("delta y : ", delta_y)

In [47]:
# Stripes file
stripes_file(
    src_input_file=src_input_file, 
    src_output=src_output, 
    tile_size_x=tile_size_x,
    tile_size_y=tile_size_y,
    )

Column 1 / 1:


100%|██████████| 5/5 [00:00<00:00,  6.10it/s]


In [50]:
# Stripes file with overlap

overlap = 5

stripes_file_with_overlap(
    src_input_file=src_input_file, 
    src_output=src_output, 
    tile_size_x=tile_size_x,
    tile_size_y=tile_size_y,
    overlap=overlap,
    )

Column 1 / 1:


100%|██████████| 5/5 [00:00<00:00,  6.15it/s]


### Merging of stripes

In [32]:
src_w_overlap = r"D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_ov_5_stripes_100_20"
src_wo_overlap = r"D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_stripes_100_20"

In [ ]:
def hash_coords(x, y, z, rounding=2):
    """Create a unique integer hash for each rounded coordinate triple."""
    return np.round(x, rounding) * 1e12 + np.round(y, rounding) * 1e6 + np.round(z, rounding)

def copy_values_attribute(src_origin, src_target, attribute_name):
    dest_src = os.path.join(os.path.dirname(src_target), 'annotated')
    os.makedirs(dest_src, exist_ok=True)

    # list_instances_src = [x for x in os.listdir(src_foler_instances_for_regrouping) if x.endswith('.laz')]
    laz_origin = laspy.read(src_origin)
    laz_target = laspy.read(src_target)
    attribute_origin = getattr(laz_origin, attribute_name)


    # Precompute hashes for the full tile once
    origin_hash = hash_coords(laz_origin.x, laz_origin.y, laz_origin.z)
    target_hash = hash_coords(laz_target.x, laz_target.y, laz_target.z)

    # Fast membership test using np.isin (vectorized)
    mask = np.isin(origin_hash, target_hash)

    # Store results
    setattr(laz_target, attribute_name, attribute_origin[mask])
    print(src_target.split('.laz')[0] + '_annotated.laz')
    laz_target.write(os.path.join(dest_src, os.path.basename(src_target)))

In [74]:
for id_file, file in enumerate(os.listdir(src_wo_overlap)):
    copy_values_attribute(
        src_origin=os.path.join(src_w_overlap, file),
        src_target=os.path.join(src_wo_overlap, file),
        attribute_name='classification',
    )
    

81425
72859
72859
(81425,)
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_0_annotated.laz
86949
70187
70187
(86949,)
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_1_annotated.laz
75247
60280
60280
(75247,)
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_2_annotated.laz
60618
48656
48656
(60618,)
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_3_annotated.laz
52872
46638
46638
(52872,)
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_4_annotated.laz


In [51]:
for id_file, file in enumerate(os.listdir(src_w_overlap)):
    print(os.path.join(src_w_overlap, file))
    laz = laspy.read(os.path.join(src_w_overlap, file))
    laz.classification[:] = 0
    laz.classification[0::10] = 1
    laz.write(os.path.join(src_w_overlap, file))

D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_ov_5_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_0.laz
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_ov_5_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_1.laz
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_ov_5_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_2.laz
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_ov_5_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_3.laz
D:\GitHubProjects\PointCloudUtils\data\SwissSurface_2019_2538_1154_tile_21_ov_5_stripes_100_20\SwissSurface_2019_2538_1154_tile_21_stripe_0_4.laz
